In [1]:
R.version.string
print("email: 'Maciek Sykulski'<macieksk@gmail.com>")
length(ls())

[1] "R version 3.3.3 (--)"

[1] "email: 'Maciek Sykulski'<macieksk@gmail.com>"


[1] 0

In [3]:
#projDir<-"/home/msykulski/msykulski/workspace/zgm/aneuploidy/"
projDir<-"/home/msykulski/workspace/genXone/hackyeah2019/"
projectPrefix<-"R_hackyeah2019"

#
load.sources<-function(){
    for (f in  c("utils.R" #Contains inpar, save_new_image, etc
                 #...  
                 ) 
        ){
    source(paste(projDir,"/src/",f,sep=""))
   }
}
load.sources()

In [4]:
#system.time(load(paste(projDir,"/r_saved_images/",projectPrefix,".Rdata",sep="")))
lsos()
gc()

,Type,Size,Rows,Columns
,<fct>,<dbl>,<dbl>,<dbl>
cpar,function,8904,NA,NA
load.sources,function,8448,NA,NA
read.table.into.db,function,7184,NA,NA
db.info,function,6200,NA,NA
multiloop,function,5376,NA,NA
postscript.call,function,4856,NA,NA
add.records.to.table,function,4704,NA,NA
save_new_image,function,4544,NA,NA
pdf.call,function,4528,NA,NA


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,534865,28.6,940480,50.3,750400,40.1
Vcells,719145,5.5,1308461,10.0,1009433,7.8


In [5]:
gc()
#system.time(save_new_image())

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,535184,28.6,940480,50.3,750400,40.1
Vcells,719097,5.5,1308461,10.0,1009433,7.8


In [41]:
library(lattice)
#library(latticeExtra)
library(ggplot2)
#library(gplots)
#library(png)

#library(Hmisc)
library(cluster)
library(MASS)
library(reshape)
library(Matrix)
library(matrixStats)
library(text2vec)
library(tokenizers)
library(doParallel)
library(parallel)

#library(lumi)
#library(lumiHumanIDMapping)
#library(lumiHumanAll.db)
#library(limma)

#library(annotate)
#library(GOstats)

#library(multtest)
#library(extrafont)
#library(enrichvs)
#library(statmod)
#library(flexmix)

load.sources()

sessionInfo()

R version 3.3.3 (--)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Debian GNU/Linux 9 (stretch)

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] doParallel_1.0.15  iterators_1.0.12   foreach_1.4.7      tokenizers_0.2.1  
 [5] text2vec_0.5.1     matrixStats_0.55.0 Matrix_1.2-8       reshape_0.8.8     
 [9] MASS_7.3-45        cluster_2.0.5      ggplot2_3.2.1      lattice_0.20-34   
[13] sqldf_0.4-11       RSQLite_2.1.2      gsubfn_0.7         proto_1.0.0       
[17] fastmatch_1.1-0    hash_2.2.6.1      

loaded via a namespace (an

In [238]:
#inpar(testp, ex( mean(rnorm(1000)) ) )
#cpar(testp)
#testp
#inpar(testp,ex({Sys.sleep(10); 1}))
#cpar(testp)
#testp
#kill.all.jobs()

[1] 0.0422968

In [8]:
data_dir<-paste0(projDir,"/jupyter_init_workspace/hackyeah_data_80/")
index<-read.table(paste0(data_dir,"INDEX"),stringsAsFactors=FALSE)

In [9]:
index_1<-read.table(paste0(data_dir,"INDEX_1"),stringsAsFactors=FALSE)
index_3<-read.table(paste0(data_dir,"INDEX_3"),stringsAsFactors=FALSE)
index_10<-read.table(paste0(data_dir,"INDEX_10"),stringsAsFactors=FALSE)

In [10]:
index_f100<-read.table(paste0(data_dir,"INDEX_F100"),stringsAsFactors=FALSE)

In [11]:
dim(index)
index[1,]

[1] 3590318       1

[1] "./501/5011c1cf5a47be37f92eb4f8a9793642f30c34b3"

In [ ]:

## https://cran.r-project.org/web/packages/text2vec/vignettes/glove.html


In [12]:
fset<-index_f100[,]
flines<-mclapply(fset,function(f)try(readLines(paste0(data_dir,f),n=-1,encoding="UTF-8",warn=FALSE)),mc.cores=44)
names(flines)<-fset
length(flines)
table(mcmapply(class,flines))      
flinesf100<-flines[mcmapply(class,flines,mc.cores=88)%fin%"character"]

[1] 70242


character 
    70242 

In [13]:
fset<-index_1[,]
flines<-mclapply(fset,function(f)try(readLines(paste0(data_dir,f),n=-1,encoding="UTF-8",warn=FALSE)),mc.cores=44)
names(flines)<-fset
length(flines)
table(mcmapply(class,flines))      
flines1<-flines[mcmapply(class,flines,mc.cores=44)%fin%"character"]

[1] 35903


character try-error 
    28736      7167 

In [238]:
#summary(mcmapply(class,flines,mc.cores=44)%fin%"character")
length(flines1)
length(flinesf100)

[1] 28736

[1] 70242

In [ ]:
#text8_file = "~/text8"
#if (!file.exists(text8_file)) {
#  download.file("http://mattmahoney.net/dc/text8.zip", "~/text8.zip")
#  unzip ("~/text8.zip", files = "text8", exdir = "~/")
#}
#wiki = readLines(text8_file, n = 1, warn = FALSE)

In [74]:
#

In [239]:
flines<-flinesf100
#tokens<-mclapply(flines,space_tokenizer,mc.cores=88)
tokens<-mclapply(flines,word_tokenizer,mc.cores=88)
#space_tokenizer(flines[[2]])
tokens_org<-tokens

In [243]:
tokens[1]
tokens[[1]][[1]]
#tolower(tokens[[1]][[5]])

$`000/0005c683654837f93fff74085799d16c40782f3c`
$`000/0005c683654837f93fff74085799d16c40782f3c`[[1]]
[1] "2018.06"  "HEAD"     "my"       "array"    "3,7,9,11" "say"      "array"   
[8] "0,1"      "2"

[1] "2018.06"  "HEAD"     "my"       "array"    "3,7,9,11" "say"      "array"   
[8] "0,1"      "2"

In [108]:
#eoftline<-rep("__GX1EOF__",5)
#tokens_l<-mclapply(tokens_org,function(tk)c(lapply(tk,tolower),list(eoftline)))

In [329]:
eoftline<-rep("__GX1EOF__",5)
tokens<-mclapply(tokens_org,function(tk)c(do.call(c,lapply(tk,function(t)c(t,'\n'))),eoftline))
length(tokens)==length(tokens_org)
head(names(tokens))

In [353]:
#sum(unlist(mclapply(tokens_org,function(tk)lapply(tk,length),mc.cores=44)))

#sum(unlist(mclapply(tokens_org,length,mc.cores=44))) #number of lines

In [253]:
#tokens_fvec<-as.list(unlist(tokens))
#tokens_fdf<-as.data.frame(do.call(rbind,mclapply(1:length(tokens),function(i)cbind(names(tokens)[i],unlist(tokens[[i]])))))

In [20]:
tokens[[1]]

[[1]]
[1] "show"      "global"    "variables" "like"      "timeout"  

[[2]]
character(0)

[[3]]
[1] "net_read_timeout" "30"              

[[4]]
character(0)

[[5]]
[1] "mysql"            "set"              "global"           "net_read_timeout"
[5] "120"             

[[6]]
 [1] "error"     "1227"      "42000"     "access"    "denied"    "you"      
 [7] "need"      "at"        "least"     "one"       "of"        "the"      
[13] "super"     "privilege" "s"         "for"       "this"      "operation"

[[7]]
[1] "__GX1EOF__" "__GX1EOF__" "__GX1EOF__" "__GX1EOF__" "__GX1EOF__"

In [ ]:
#head(tokens_fvec,3)

In [111]:
#head(tokens_fdf)
#class(tokens_fdf$V1) #<-factor(tokens_fdf$V1)
#class(tokens_fdf$V2) #<-factor(tokens_fdf$V2)
#dim(tokens_fdf)

In [118]:
#tokens_fdf$V2[tokens_fdf$V1==
#tokens<-do.call(c,tokens_l)
#names(tokens)<-NULL

In [254]:
#head(tokens,10)
#tokens2<-do.call(c,mclapply(tokens,as.list))

In [255]:
#class(tokens2)
#head(tokens2)

In [256]:
#class(unlist(tokens2))

In [67]:
#which(names(tokens)=="00c/00c1a6d1344389ca2ad5bfdb97759c944eebd696996")
#tokens[names(tokens)%fin%as.character(tokens_fdf$V1[tokens_fdf$V2=="dirty"][10])]

#class(head(names(tokens)))

In [257]:
#tokens3<-unlist(tokens2)
#head(tokens3,40)
#length(tokens3)

In [354]:
#it <- itoken_parallel(tokens, progressbar = FALSE,n_chunks=20)
it <- itoken(tokens, progressbar = FALSE,n_chunks=20)
it

<itoken>
  Inherits from: <iterator>
  Public:
    chunk_size: 3513
    clone: function (deep = FALSE) 
    counter: 0
    ids: 000/0005c683654837f93fff74085799d16c40782f3c 000/0004579 ...
    initialize: function (iterable, ids = NULL, n_chunks = 10, progress_ = interactive(), 
    is_complete: active binding
    iterable: list
    length: active binding
    nextElem: function () 
    preprocessor: list
    progress: FALSE
    progressbar: NULL
    tokenizer: list

In [351]:
#cl <- makeCluster(88)
#registerDoParallel(cl)
#system.time(tcm <- create_tcm(it, vectorizer, skip_grams_window = 13L))

In [26]:
#names(tokens_fvec)[tokens_fvec=="paris"]
#length(tokens_fvec)

In [191]:
#head(tokens,10000)

In [75]:
#which("./1a9/1a9af5ba2e42ac520f69ccd88a68abd7505b59"==flines)
#head(flines,2)

$`./1a9/1a9af5ba2e42ac520f69ccd88a68abd7505b59ed`
[1] "show global variables like '%timeout%';"                                                                
[2] "\t"                                                                                                     
[3] "| net_read_timeout                          | 30       |"                                               
[4] "\t"                                                                                                     
[5] "mysql> SET GLOBAL net_read_timeout=120;"                                                                
[6] "ERROR 1227 (42000): Access denied; you need (at least one of) the SUPER privilege(s) for this operation"

$`./0a7/0a77fd7fb6426d864977a9322f8df17aeeeda2de`
 [1] "#!/usr/bin/env ruby"                                        
 [2] "# -*- coding: UTF-8 -*-"                                    
 [3] ""                                                           
 [4] "#"                                                          
 [5] "# $ ruby -v"                                                
 [6] "# ruby 1.9.3p194 (2012-04-20 revision 35410) [x86_64-linux]"
 [7] "#"                                                          
 [8] "x = \"xyz\""                                                
 [9] ""                                                           
[10] "p %!abc#{x}!"                                               
[11] "p %\"abc#{x}\""                                             
[12] "#p %#abc#{x}#   ### Syntax Error"                           
[13] "p %$abc#{x}$"                                               
[14] "p %%abc#{x}%"                                               
[15] "p %&abc#{x}&"                                               
[16] "p %'abc#{x}'"                                               
[17] "p %(abc#{x})"                                               
[18] "p %)abc#{x})"                                               
[19] "p %*abc#{x}*"                                               
[20] "p %+abc#{x}+"                                               
[21] "p %,abc#{x},"                                               
[22] "p %-abc#{x}-"                                               
[23] "p %.abc#{x}."                                               
[24] "p %/abc#{x}/"                                               
[25] "p %:abc#{x}:"                                               
[26] "p %;abc#{x};"                                               
[27] "p %<abc#{x}>"                                               
[28] "#p %=abc#{x}=   ### NG"                                     
[29] "p %>abc#{x}>"                                               
[30] "p %?abc#{x}?"                                               
[31] "p %@abc#{x}@"                                               
[32] "p %[abc#{x}]"                                               
[33] "p %\\abc#{x}\\"                                             
[34] "p %]abc#{x}]"                                               
[35] "p %^abc#{x}^"                                               
[36] "p %_abc#{x}_"                                               
[37] "p %`abc#{x}`"                                               
[38] "p %{abc#{x}}"                                               
[39] "p %|abc#{x}|"                                               
[40] "p %}abc#{x}}"                                               
[41] "p %~abc#{x}~"                                               
[42] ""                                                           
[43] "p ( %!abc#{x}! )"                                           
[44] "p ( %\"abc#{x}\" )"                                         
[45] "#p ( %#abc#{x}# )   ### Syntax Error"                       
[46] "p ( %$abc#{x}$ )"                                           
[47] "p ( %%abc#{x}% )"                                           
[48] "p ( %&abc#{x}& )"                                           
[49] "p ( %'abc#{x}' )"

In [375]:
tokens[which(mcmapply(function(tk)"paris"%in%tk&"france"%in%tk,tokens))]

$`018/018c61ace91c897813746f977820a630a53a027d`
   [1] "res.setHeader"                           
   [2] "Content"                                 
   [3] "Type"                                    
   [4] "application"                             
   [5] "rss"                                     
   [6] "xml"                                     
   [7] "\n"                                      
   [8] "const"                                   
   [9] "myXml"                                   
  [10] "\n"                                      
  [11] "xml"                                     
  [12] "version"                                 
  [13] "1.0"                                     
  [14] "encoding"                                
  [15] "utf"                                     
  [16] "8"                                       
  [17] "\n"                                      
  [18] "rss"                                     
  [19] "xmlns:media"                             
  [20] "http"                                    
  [21] "search.yahoo.com"                        
  [22] "mrss"                                    
  [23] "xmlns:dc"                                
  [24] "http"                                    
  [25] "purl.org"                                
  [26] "dc"                                      
  [27] "elements"                                
  [28] "1.1"                                     
  [29] "version"                                 
  [30] "2.0"                                     
  [31] "\n"                                      
  [32] "channel"                                 
  [33] "\n"                                      
  [34] "title"                                   
  [35] "The"                                     
  [36] "Guardian"                                
  [37] "title"                                   
  [38] "\n"                                      
  [39] "link"                                    
  [40] "https"                                   
  [41] "www.theguardian.com"                     
  [42] "international"                           
  [43] "link"                                    
  [44] "\n"                                      
  [45] "description"                             
  [46] "Latest"                                  
  [47] "international"                           
  [48] "news"                                    
  [49] "sport"                                   
  [50] "and"                                     
  [51] "comment"                                 
  [52] "from"                                    
  [53] "the"                                     
  [54] "Guardian"                                
  [55] "description"                             
  [56] "\n"                                      
  [57] "language"                                
  [58] "en"                                      
  [59] "gb"                                      
  [60] "language"                                
  [61] "\n"                                      
  [62] "copyright"                               
  [63] "Guardian"                                
  [64] "News"                                    
  [65] "and"                                     
  [66] "Media"                                   
  [67] "Limited"                                 
  [68] "or"                                      
  [69] "its"                                     
  [70] "affiliated"                              
  [71] "companies"                               
  [72] "All"                                     
  [73] "rights"                                  
  [74] "reserved"                                
  [75] "2018"                                    
  [76] "copyright"                               
  [77] "\n"                                      
  [78] "pubDate"                                 
  [79] "Tue"                                     
  

In [355]:
#create_vocabulary
RcppParallel::setThreadOptions(numThreads = 80) ##Doesn't seem to improve much here
system.time(vocab <- create_vocabulary(it))
vocab_all<-vocab

   user  system elapsed 
  7.778   0.123   7.902 

In [356]:
#?prune_vocabulary
#prune_vocabulary(vocabulary, term_count_min = 1L, term_count_max = Inf,
#  doc_proportion_min = 0, doc_proportion_max = 1, doc_count_min = 1L,
#  doc_count_max = Inf, vocab_term_max = Inf)
head(vocab_all)
dim(vocab_all)

,term,term_count,doc_count
,<chr>,<int>,<int>
2,DG699QNY,1,1
3,froterujmyż,1,1
5,Borno,1,1
6,DCN,1,1
7,LeveledWeapons,1,1
8,dataArray.sort,1,1


[1] 1952529       3

In [357]:
vocab <- prune_vocabulary(vocab_all, term_count_min = 5L)
#vocab <- prune_vocabulary(vocab_all, term_count_min = 5L)
head(vocab)
dim(vocab)

,term,term_count,doc_count
,<chr>,<int>,<int>
62,03T071501,5,5
85,Sprog,5,2
115,DST_IMG,5,1
117,Hat.Parent,5,5
143,nears,5,4
153,astarwar,5,2


[1] 239862      3

In [358]:
vocab[vocab$term=="paris",]

,term,term_count,doc_count
,<chr>,<int>,<int>
1554424,paris,19,15


In [360]:
# Use our filtered vocabulary
vocab<-vocab_all
vectorizer <- vocab_vectorizer(vocab)

In [371]:
# use window of 5 for context words
#cl <- makeCluster(88)
#registerDoParallel(cl)
system.time(tcm <- create_tcm(it, vectorizer, 
                              skip_grams_window = 50L, weights = sqrt(1/seq_len(skip_grams_window))))

   user  system elapsed 
176.743   4.458 181.206 

In [372]:
sum(tcm)

[1] 116630773

In [159]:
#system.time(tcml <- mclapply(it,function(itt)create_tcm(itt, vectorizer, skip_grams_window = 13L),mc.cores=88))
#tcm<-tcml[[1]]
#for (t in tcml[-1]){tcm<-tcm+t}
#rownames(tcm)<-rownames(tcml[[1]])
#colnames(tcm)<-colnames(tcml[[1]])

In [374]:
tcm[1:10,1:10]
#tcml[[1]][1:10,1:10]
sum(tcm)

   [[ suppressing 10 column names ‘DG699QNY’, ‘froterujmyż’, ‘Borno’ ... ]]


10 x 10 sparse Matrix of class "dgTMatrix"
                                   
DG699QNY        . . . . . . . . . .
froterujmyż     . . . . . . . . . .
Borno           . . . . . . . . . .
DCN             . . . . . . . . . .
LeveledWeapons  . . . . . . . . . .
dataArray.sort  . . . . . . . . . .
scudzoziemczaną . . . . . . . . . .
fogel           . . . . . . . . . .
SI09gvsCAOgE9v  . . . . . . . . . .
nieśródukładowi . . . . . . . . . .

[1] 116630773

In [373]:
x<-tcm[which(rownames(tcm)=="dirty"),]
tcm[which(rownames(tcm)=="dirty"),x!=0]

dirty            writer             flesh               Pop 
       9.37659931        0.07142857        0.54000002        0.56128395 
      declaration               dec            remain             sight 
       0.17692308        0.02173913        0.02857143        0.02500000 
            fresh             heads        activation          Skipping 
       0.15000001        0.02439024        0.06644880        0.03448276 
            loves               aka             stuck               Kay 
       0.07142857        0.58333331        0.43754452        0.03846154 
        displayed            animal             walks           realize 
       0.04000000        0.06475486        0.07083333        0.36111110 
            Still           knowing           imagine           despite 
       2.07142854        0.08388158        0.29090911        0.02325581 
              ill             apart            beauty        associated 
       0.33333334        0.16666667        0.03703704        0.02941176 
            laugh               187          whenever           passing 
       0.45535713        0.03846154        0.06573276        0.05263158 
        searching            cookie                km          handling 
       0.02325581        0.25000000        0.08333334        1.00000000 
          Average            travel              Fuck              Mail 
       0.02439024        0.06573276        0.11904763        0.03225806 
              sir          possibly            Legend              LINK 
       0.03333334        0.06787331        0.02222222        0.02173913 
            finds             trash             blade          Released 
       0.33333334        0.03571429        0.53333336        0.03571429 
            sweet            former              suoi              poor 
       0.11111111        0.25000000        0.03703704        0.25000000 
             ECHO             penis           helping             stood 
       0.02272727        0.05882353        0.05555556        0.11111111 
          sitting        coordinate             you'd          reaction 
       0.04347826        0.07349956        0.02380952        0.11212122 
        certainly        throughout              luck         temporary 
       0.02040816        0.20000000        0.13221154        0.33333334 
         pleasure           Captain            issued           uniform 
       0.15879443        0.08608059        0.10000000        0.06777778 
              tar           stories              THAT               HAS 
       0.16666667        0.04166667        0.20000000        0.10250000 
          turning            RETURN             kinda      professional 
       0.09775642        0.08545455        0.02631579        0.03703704 
             spam              Bill            Visual              Over 
       0.14285715        0.28448278        0.03571429        0.07204969 
             gift          practice           showing             Queen 
       0.05882353        0.02500000        0.10768253        0.05594406 
              sun          modified         continued              Many 
       0.03703704        0.12500000        0.16111112        0.12500000 
          clothes                ko        attempting                af 
       0.09444445        0.02000000        0.26439396        0.05555556 
         involved         according        Attributes            silver 
       0.14973262        0.03125000        1.43826771        0.02380952 
             repo              fuel        a263b55282             tight 
       0.08333334        0.10000000        0.02000000        0.17936912 
             warm              heal                Am               Art 
       0.04166667        0.02040816        0.38535354        0.10000000 
            Chris           smaller             spent               san 
       0.17907415        0.02127660        0.08055556        0.10131332 
           tongue           clearly            beyond             Chai

In [297]:
glove <- GlobalVectors$new(word_vectors_size = 30, learning_rate = 0.10, vocabulary = vocab, x_max = 10)

In [298]:
RcppParallel::setThreadOptions(numThreads = 88)
system.time(glove$fit_transform(tcm, n_iter = 180))
# `glove` object will be modified by `fit()` call !
#fit(tcm, glove, n_iter = 20)

INFO [2019-09-15 01:26:34] 2019-09-15 01:26:34 - epoch 1, expected cost 0.1200
INFO [2019-09-15 01:26:35] 2019-09-15 01:26:35 - epoch 2, expected cost 0.0871
INFO [2019-09-15 01:26:36] 2019-09-15 01:26:36 - epoch 3, expected cost 0.0784
INFO [2019-09-15 01:26:37] 2019-09-15 01:26:37 - epoch 4, expected cost 0.0734
INFO [2019-09-15 01:26:37] 2019-09-15 01:26:37 - epoch 5, expected cost 0.0700
INFO [2019-09-15 01:26:38] 2019-09-15 01:26:38 - epoch 6, expected cost 0.0676
INFO [2019-09-15 01:26:39] 2019-09-15 01:26:39 - epoch 7, expected cost 0.0657
INFO [2019-09-15 01:26:40] 2019-09-15 01:26:40 - epoch 8, expected cost 0.0642
INFO [2019-09-15 01:26:41] 2019-09-15 01:26:40 - epoch 9, expected cost 0.0630
INFO [2019-09-15 01:26:41] 2019-09-15 01:26:41 - epoch 10, expected cost 0.0620
INFO [2019-09-15 01:26:42] 2019-09-15 01:26:42 - epoch 11, expected cost 0.0611
INFO [2019-09-15 01:26:43] 2019-09-15 01:26:43 - epoch 12, expected cost 0.0603
INFO [2019-09-15 01:26:44] 2019-09-15 01:26:44 - 

    user   system  elapsed 
8538.703   11.013  118.542 

In [300]:
#system.time(word_vectors <- glove$get_word_vectors())
word_vectors <- glove$components

In [301]:
dim(word_vectors)

word_vectors[,"dirty"]

[1]     30 103984

[1] -0.00698050 -0.10016550  0.16785847  0.29412934 -0.31369671 -0.06220389
 [7] -0.27294591  0.12394971  0.16278943 -0.06936778 -0.45542809 -0.20109227
[13] -0.05991666  0.14226310 -0.31946549 -0.66026819 -0.38823104  0.54345447
[19] -0.20275904 -0.42743903  0.11677223  0.32164136 -0.83988243  0.52802593
[25]  0.04181979 -0.42763278 -0.49631199 -0.23519617  0.37234718 -0.38671234

In [177]:
summary(colSums(word_vectors))

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-10.9900  -1.5060   0.1635   0.1617   1.8480  12.3600 

In [178]:
head(colnames(word_vectors),20)

[1] "peninsula"       "出力"            "scorpio"         "klanu"          
 [5] "equivalente"     "0ubuntu1.1"      "arequest"        "krev"           
 [9] "current_item"    "3.1415"          "vært"            "psz"            
[13] "heartbreak"      "killcounter"     "38685"           "dystopia"       
[17] "capson"          "py36hfa6e2cd_1"  "db11g"           "leitor.nextline"

In [296]:
dim(word_vectors)
dim(sim_dirty)

[1]    100 103984

[1] 103984      1

In [376]:
#sim_dirty<-sim2(t(word_vectors),t(word_vectors[,"usa",drop=FALSE]),method="cosine",norm="l2")
w<-word_vectors[,"king",drop=FALSE]-word_vectors[,"man",drop=FALSE]+word_vectors[,"woman",drop=FALSE]
#sim_dirty<-cor(word_vectors,word_vectors[,"king",drop=FALSE]) #,method="cosine",norm="l2")
sim_dirty<-cor(word_vectors,w) #,method="cosine",norm="l2")


In [377]:
sim_dirty[order(sim_dirty[,1],decreasing=TRUE)[1:50],]

king            knot            19th           queen            meal 
      0.7754113       0.7190819       0.7076692       0.6992938       0.6986500 
         melted      occasional         schultz            21st             9th 
      0.6953629       0.6872420       0.6830186       0.6783149       0.6752939 
         03.823          forest           coast        dinosaur          fallen 
      0.6725494       0.6723289       0.6675002       0.6672879       0.6640970 
      raven1907 jsonparser.java          izvrsi         dwarves system.data.dll 
      0.6629443       0.6610905       0.6599330       0.6585459       0.6581220 
        ruffled mutablelivedata        brojdana        weirdest            20th 
      0.6543556       0.6469763       0.6458574       0.6457754       0.6437053 
           18th         poisons       naturally         battles             ing 
      0.6414993       0.6410872       0.6400489       0.6393829       0.6387249 
      legendary            orbs   consciousness        01110100         victory 
      0.6377336       0.6362200       0.6340913       0.6333651       0.6330973 
        empires          ransom          48.521          facted          coffin 
      0.6326237       0.6317532       0.6273019       0.6264276       0.6215348 
        keywait     levaya_ryka         bullies         pigeons          debuff 
      0.6209411       0.6207070       0.6196159       0.6196009       0.6184264 
          fifth            16th          colder          emails        00100000 
      0.6160648       0.6158357       0.6148397       0.6146680       0.6146679

In [379]:
##Code from old example, not working
#berlin <- word_vectors["paris", , drop = FALSE] - 
#  word_vectors["france", , drop = FALSE] + 
#  word_vectors["germany", , drop = FALSE]

[1] 71977128

[1] 71985240

[1] 5198672

[1] 5199705

In [22]:
#diff(as.integer(df_5w15$bit_score_15),as.integer(df_5w10$bit_score_10))
#sum(as.integer(df$bit_score_15)+as.integer(df$bit_score_10))
df<-transform(df, dif_mean=abs(round(((mean_bit_score_15-mean_bit_score_10)/(mean_bit_score_15+mean_bit_score_10)*100),5)))
df<-transform(df, dif_median=abs(round(((median_bit_score_15-median_bit_score_10)/(median_bit_score_15+median_bit_score_10)*100),5)))
df


,taxid,count_10,mean_bit_score_10,median_bit_score_10,mean_len_10,median_len_10,count_15,mean_bit_score_15,median_bit_score_15,mean_len_15,median_len_15,dif_mean,dif_median
263,91061,2,283.0000,283.00,411.0000,411.0,1,100.0000,100.0,58.00000,58.0,47.78068,47.78068
395,338193,1,279.0000,279.00,504.0000,504.0,2,200.0000,200.0,415.00000,415.0,16.49269,16.49269
41,1302,3,1217.6667,1275.00,1239.0000,1379.0,2,977.5000,977.5,1139.00000,1139.0,10.94070,13.20755
265,94868,12,663.9167,557.50,780.6667,651.5,11,533.1818,298.0,619.00000,366.0,10.92098,30.33314
252,84037,14,498.3571,377.50,740.5714,658.0,12,407.4167,269.0,645.83333,299.5,10.04009,16.78268
176,37659,8,729.2500,841.00,925.5000,1133.5,7,611.4286,640.0,824.00000,956.0,8.78819,13.57191
139,29364,3,822.3333,767.00,1228.0000,1439.0,4,719.7500,688.5,1102.75000,1116.5,6.65226,5.39334
181,38304,5,1213.6000,1175.00,1164.4000,1316.0,4,1068.0000,1021.0,1118.00000,1232.0,6.38149,7.01275
413,371674,4,692.2500,664.50,1044.7500,1178.5,3,612.0000,518.0,1007.00000,1199.0,6.15296,12.38901
617,1506553,39,903.0000,1011.00,1082.3077,1337.0,34,816.4118,901.5,1005.02941,1243.0,5.03592,5.72549


In [25]:
df_mean <- df[order(-df$dif_mean),]
df_median <- df[order(-df$dif_median),]

In [44]:
a <- df_mean[1:20,]
a <- rbind(a,df_median[1:20,])
unique(a)
unique(a[,1])

,taxid,count_10,mean_bit_score_10,median_bit_score_10,mean_len_10,median_len_10,count_15,mean_bit_score_15,median_bit_score_15,mean_len_15,median_len_15,dif_mean,dif_median
599,1432052,1,158.0000,158.0,132.0000,132.0,2,500.5000,500.5,726.5000,726.5,52.01215,52.01215
263,91061,2,283.0000,283.0,411.0000,411.0,1,100.0000,100.0,58.0000,58.0,47.78068,47.78068
127,28221,1,135.0000,135.0,164.0000,164.0,2,218.5000,218.5,315.0000,315.0,23.62093,23.62093
395,338193,1,279.0000,279.0,504.0000,504.0,2,200.0000,200.0,415.0000,415.0,16.49269,16.49269
634,1648214,1,549.0000,549.0,773.0000,773.0,2,760.5000,760.5,974.0000,974.0,16.15120,16.15120
108,1730,1,193.0000,193.0,306.0000,306.0,2,244.5000,244.5,399.0000,399.0,11.77143,11.77143
41,1302,3,1217.6667,1275.0,1239.0000,1379.0,2,977.5000,977.5,1139.0000,1139.0,10.94070,13.20755
265,94868,12,663.9167,557.5,780.6667,651.5,11,533.1818,298.0,619.0000,366.0,10.92098,30.33314
252,84037,14,498.3571,377.5,740.5714,658.0,12,407.4167,269.0,645.8333,299.5,10.04009,16.78268
36,1267,8,557.2500,561.0,677.7500,698.0,9,666.7778,566.0,754.8889,770.0,8.94814,0.44366


[1] 1432052   91061   28221  338193 1648214    1730    1302   94868   84037
[10]    1267   37659   29347   29364   38304  371674 1211817 1506553 1096246
[19] 1157312 1720294    1305  115544    1561   33034  290052